In [1]:
# Corrección de cuantiles empírica.

import os
import sys

import pandas as pd
import numpy  as np

import xarray as xr

In [2]:
# Rutas de archivos.

names = [ "NSRDB", "NSRDB_2" ]
lat   = 19.41
lon   = -99.14

#df_d = xr.open_dataset(f"results/{names[0]}.nc").sel(
#    {"lat": lat, "lon": lon} ).to_dataframe().reset_index(
#    ).drop( ["lat", "lon"], axis = 1 ).set_index("time")

path_res = "../results/"
path_d = [ f"{path_res}{names[0]}.nc",
    f"{path_res}{names[1]}.nc" ]
path_r = f"{path_res}{names[1]}_{lat}_{lon}_quantile.nc"

df_d = xr.open_dataset( f"{path_d[0]}" ).sel(
    {"lat": lat, "lon": lon} ).to_dataframe(
    ).reset_index().drop( ["lon", "lat"], axis = 1 ).set_index("time")
df_r = xr.open_dataset( f"{path_d[1]}" ).sel(
    {"lat": lat, "lon": lon} ).to_dataframe(
    ).reset_index().drop( ["lon", "lat"], axis = 1 ).set_index("time")

In [43]:
# Creamos DataFrames de apoyo.
df_m = df_r.copy()
df_q = pd.DataFrame( index = np.linspace( 1/df_r.shape[0], 1, df_r.shape[0] ),
    columns = df_r.columns )
df_q = df_q.rename_axis("CDF", axis = 0)

# Inicializamos las variables como 0.
df_m[ ["GHI", "DNI"] ] = 0

# Iteramos para todas las columnas.
for v in df_d.columns:
    # Ordenamos los valores originales y destino incluyendo el tiempo.
    if v in ["DNI", "GHI"]:
        df_i = df_d[[v]].where( df_d[[v]] > 0, np.nan
            ).dropna().sort_values(v).reset_index()
        df_j = df_r[[v]].where( df_r[[v]] > 0, np.nan
            ).dropna().sort_values(v).reset_index()
    else:
        df_i = df_d[[v]].sort_values(v).reset_index()
        df_j = df_r[[v]].sort_values(v).reset_index()
    # Calculamos la distribución acumulada.
    df_i["CDF"] = np.linspace( 1/df_i.shape[0], 1, df_i.shape[0] )
    df_j["CDF"] = np.linspace( 1/df_j.shape[0], 1, df_j.shape[0] )

    # Interpolamos los valores de la distribución origen a la destino.
    df_j["Map"] = np.interp( df_j["CDF"], df_i["CDF"],
        df_i[v] ).round( decimals = 2 )
    df_j["Quantile"] =  df_j["Map"] - df_j[v]
    df_j = df_j.set_index("time").sort_index()
    # Reordenamos los valores.
    df_m.loc[ df_j.index, v ] = df_j["Map"]
    # Guardamos el mapeo de cuantiles para la corrida futura.
    df_j = df_j.set_index("CDF").sort_index()
    if v in ["DNI", "GHI"]:
          q = np.interp( df_q.index, df_j.index, df_j["Quantile"].values )
    else: q = df_j["Quantile"].values
    df_q[v] = q

# Nos aseguramos que los datos estén dentro de su rango.
df_m["Relative Humidity"] = df_m["Relative Humidity"].where(
    df_m["Relative Humidity"] > 0, 0 )
df_m["Relative Humidity"] = df_m["Relative Humidity"].where(
    df_m["Relative Humidity"] < 100, 100 )
df_m["Wind Direction"] = df_m["Wind Direction"].where(
    df_m["Wind Direction"] < 360, df_m["Wind Direction"] - 360 )
df_m["Wind Speed"] = df_m["Wind Speed"].where( df_m["Wind Speed"] > 0 , 0 )
df_m["GHI"] = df_m["GHI"].where( df_m["GHI"] > 0 , 0 )
df_m["DNI"] = df_m["DNI"].where( df_m["DNI"] > 0 , 0 )

df_m

,Temperature,GHI,Relative Humidity,Pressure,Wind Speed,Wind Direction,DNI
time,,,,,,,
2006-01-01 00:00:00,11.7,0.0,68.7,785.0,0.7,196.0,0.0
2006-01-01 01:00:00,11.0,0.0,69.8,784.0,0.8,208.0,0.0
2006-01-01 02:00:00,10.8,0.0,69.2,784.0,0.8,221.0,0.0
2006-01-01 03:00:00,10.4,0.0,68.4,784.0,0.6,232.0,0.0
2006-01-01 04:00:00,10.0,0.0,68.0,784.0,0.6,248.0,0.0
...,...,...,...,...,...,...,...
2022-12-31 19:00:00,12.4,0.0,54.6,785.0,0.9,250.0,0.0
2022-12-31 20:00:00,11.5,0.0,59.4,786.0,1.1,255.0,0.0
2022-12-31 21:00:00,11.3,0.0,60.6,786.0,0.8,261.0,0.0
